In [1]:
using SDDP, HiGHS

model = SDDP.LinearPolicyGraph(;
    stages = 3,
    sense = :Min,
    lower_bound = 0.0,
    optimizer = HiGHS.Optimizer,
) do subproblem, node
    # State variables
    @variable(subproblem, 0 <= volume <= 200, SDDP.State, initial_value = 200)
    # Control variables
    @variables(subproblem, begin
        thermal_generation >= 0
        hydro_generation >= 0
        hydro_spill >= 0
    end)
    # Random variables
    @variable(subproblem, inflow)
    Ω = [0.0, 50.0, 100.0]
    P = [1 / 3, 1 / 3, 1 / 3]
    SDDP.parameterize(subproblem, Ω, P) do ω
        return JuMP.fix(inflow, ω)
    end
    # Transition function and constraints
    @constraints(
        subproblem,
        begin
            volume.out == volume.in - hydro_generation - hydro_spill + inflow
            demand_constraint, hydro_generation + thermal_generation == 150
        end
    )
    # Stage-objective
    if node == 1
        @stageobjective(subproblem, 50 * thermal_generation)
    elseif node == 2
        @stageobjective(subproblem, 100 * thermal_generation)
    else
        @assert node == 3
        @stageobjective(subproblem, 150 * thermal_generation)
    end
end

A policy graph with 3 nodes.
 Node indices: 1, 2, 3


In [2]:
SDDP.train(model; iteration_limit = 5)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-24
-------------------------------------------------------------------
problem
  nodes           : 3
  state variables : 1
  scenarios       : 2.70000e+01
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [7, 7]
  AffExpr in MOI.EqualTo{Float64}         : [2, 2]
  VariableRef in MOI.GreaterThan{Float64} : [5, 5]
  VariableRef in MOI.LessThan{Float64}    : [1, 2]
numerical stability report
  matrix range     [1e+00, 1e+00]
  objective range  [1e+00, 2e+02]
  bounds range     [2e+02, 2e+02]
  rhs range        [2e+02, 2e+02]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-----------------------------------------------

In [3]:
simulations = SDDP.simulate(
    # The trained model to simulate.
    model,
    # The number of replications.
    100,
    # A list of names to record the values of.
    [:volume, :thermal_generation, :hydro_generation, :hydro_spill],
)

replication = 1
stage = 2
simulations[3][1]

Dict{Symbol, Any} with 10 entries:
  :volume             => State{Float64}(200.0, 200.0)
  :hydro_spill        => 0.0
  :bellman_term       => 3333.33
  :noise_term         => 100.0
  :node_index         => 1
  :stage_objective    => 2500.0
  :objective_state    => nothing
  :thermal_generation => 50.0
  :hydro_generation   => 100.0
  :belief             => Dict(1=>1.0)

In [4]:
print(model[2].subproblem)

In [14]:
using SDDP, HiGHS

Ω = [
    (inflow = 0.0, fuel_multiplier = 1.5),
    (inflow = 50.0, fuel_multiplier = 1.0),
    (inflow = 100.0, fuel_multiplier = 0.75),
]

model = SDDP.MarkovianPolicyGraph(;
    transition_matrices = Array{Float64,2}[
        [1.0]',
        [0.75 0.25],
        [0.75 0.25; 0.25 0.75],
    ],
    sense = :Min,
    lower_bound = 0.0,
    optimizer = HiGHS.Optimizer,
) do subproblem, node
    # Unpack the stage and Markov index.
    t, markov_state = node
    # Define the state variable.
    @variable(subproblem, 0 <= volume <= 200, SDDP.State, initial_value = 200)
    # Define the control variables.
    @variables(subproblem, begin
        thermal_generation >= 0
        hydro_generation >= 0
        hydro_spill >= 0
        inflow
    end)
    # Define the constraints
    @constraints(
        subproblem,
        begin
            volume.out == volume.in + inflow - hydro_generation - hydro_spill
            thermal_generation + hydro_generation == 150.0
        end
    )
    # Note how we can use `markov_state` to dispatch an `if` statement.
    probability = if markov_state == 1  # wet climate state
        [1 / 6, 1 / 3, 1 / 2]
    else  # dry climate state
        [1 / 2, 1 / 3, 1 / 6]
    end

    fuel_cost = [50.0, 100.0, 150.0]
    SDDP.parameterize(subproblem, Ω, probability) do ω
        JuMP.fix(inflow, ω.inflow)
        @stageobjective(
            subproblem,
            ω.fuel_multiplier * fuel_cost[t] * thermal_generation
        )
    end
end

A policy graph with 5 nodes.
 Node indices: (1, 1), (2, 1), (2, 2), (3, 1), (3, 2)


In [15]:
SDDP.train(model, iteration_limit= 10)

-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-24
-------------------------------------------------------------------
problem
  nodes           : 5
  state variables : 1
  scenarios       : 1.08000e+02
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [7, 7]
  AffExpr in MOI.EqualTo{Float64}         : [2, 2]
  VariableRef in MOI.GreaterThan{Float64} : [5, 5]
  VariableRef in MOI.LessThan{Float64}    : [1, 2]
numerical stability report
  matrix range     [1e+00, 1e+00]
  objective range  [1e+00, 2e+02]
  bounds range     [2e+02, 2e+02]
  rhs range        [2e+02, 2e+02]
-------------------------------------------------------------------
 iteration    simulation      bound        time (s)     solves  pid
-----------------------------------------------

In [7]:
fieldnames(typeof(model[(1,1)]))

(:index, :subproblem, :children, :noise_terms, :parameterize, :states, :stage_objective, :stage_objective_set, :bellman_function, :objective_state, :belief_state, :pre_optimize_hook, :post_optimize_hook, :has_integrality, :optimizer, :ext, :lock)

In [12]:
#(:cut_type, :global_theta, :local_thetas, :risk_set_cuts)

#(:theta, :states, :objective_states, :belief_states, :cuts, :sampled_states, :cuts_to_be_deleted, :deletion_minimum)

#(:intercept, :coefficients, :obj_y, :belief_y, :non_dominated_count, :constraint_ref)


model[(1,1)].bellman_function.global_theta.sampled_states

3-element Vector{SDDP.SampledState}:
 SDDP.SampledState(Dict(:volume => 0.0), nothing, nothing, SDDP.Cut(23177.083333333332, Dict(:volume => -157.8125), nothing, nothing, 1, 157.8125 volume_out + _[7] ≥ 23177.083333333332), 23177.083333333332)
 SDDP.SampledState(Dict(:volume => 146.86468646864685), nothing, nothing, SDDP.Cut(22604.166666666664, Dict(:volume => -95.05208333333333), nothing, nothing, 2, 95.05208333333333 volume_out + _[7] ≥ 22604.166666666664), 8644.372249724973)
 SDDP.SampledState(Dict(:volume => 200.0), nothing, nothing, SDDP.Cut(22604.166666666664, Dict(:volume => -95.05208333333333), nothing, nothing, 2, 95.05208333333333 volume_out + _[7] ≥ 22604.166666666664), 3593.75)

In [166]:
using SDDP, HiGHS, Test

S = [  # cutting, stage
    0 1 2
    0 0 1
    0 0 0
]
t = [60, 60, 245]  # days in period
D = [210, 210, 858]  # demand
q = [  # selling price per bale
    [4.5 4.5 4.5; 4.5 4.5 4.5; 4.5 4.5 4.5],
    [5.5 5.5 5.5; 5.5 5.5 5.5; 5.5 5.5 5.5],
    [6.5 6.5 6.5; 6.5 6.5 6.5; 6.5 6.5 6.5],
]
b = [  # predicted yield (bales/acres) from cutting i in weather j.
    30 75 37.5
    15 37.5 18.25
    7.5 18.75 9.325
]
w = 3000  # max storage
C = [50 50 50; 50 50 50; 50 50 50]  # cost to grow hay
r = [  # Cost per bale of hay from cutting i during weather condition j.
    [5 5 5; 5 5 5; 5 5 5],
    [6 6 6; 6 6 6; 6 6 6],
    [7 7 7; 7 7 7; 7 7 7],
]
M = 60.0  # max acreage for planting
H = 0.0  # initial inventory
V = [0.05, 0.05, 0.05]  # inventory cost
L = 3000.0  # max demand for hay

graph = SDDP.MarkovianGraph([
    ones(Float64, 1, 1),
    [0.14 0.69 0.17],
    [0.14 0.69 0.17; 0.14 0.69 0.17; 0.14 0.69 0.17],
    [0.14 0.69 0.17; 0.14 0.69 0.17; 0.14 0.69 0.17],
])

model = SDDP.PolicyGraph(
    graph;
    lower_bound = 0.0,
    optimizer = HiGHS.Optimizer,
) do subproblem, index
    stage, weather = index
    # ===================== State Variables =====================
    # Area planted.
    @variable(subproblem, 0 <= acres <= M, SDDP.State, initial_value = M)
    @variable(
        subproblem,
        bales[i = 1:3] >= 0,
        SDDP.State,
        initial_value = (i == 1 ? H : 0)
    )
    # ===================== Variables =====================
    @variables(subproblem, begin
        buy[1:3] >= 0  # Quantity of bales to buy from each cutting.
        sell[1:3] >= 0 # Quantity of bales to sell from each cutting.
        eat[1:3] >= 0  # Quantity of bales to eat from each cutting.
        pen_p[1:3] >= 0  # Penalties
        pen_n[1:3] >= 0  # Penalties
    end)
    # ===================== Constraints =====================
    if stage == 1
        @constraint(subproblem, acres.out <= acres.in)
        @constraint(subproblem, [i = 1:3], bales[i].in == bales[i].out)
    else
        @expression(
            subproblem,
            cut_ex[c = 1:3],
            bales[c].in + buy[c] - eat[c] - sell[c] + pen_p[c] - pen_n[c]
        )
        @constraints(
            subproblem,
            begin
                # Cannot plant more land than previously cropped.
                acres.out <= acres.in
                # In each stage we need to meet demand.
                sum(eat) >= D[stage-1]
                # We can buy and sell other cuttings.
                bales[stage-1].out ==
                cut_ex[stage-1] + acres.in * b[stage-1, weather]
                [c = 1:3; c != stage - 1], bales[c].out == cut_ex[c]
                # There is some maximum storage.
                sum(bales[i].out for i in 1:3) <= w
                # We can only sell what is in storage.
                [c = 1:3], sell[c] <= bales[c].in
                # Maximum sales quantity.
                sum(sell) <= L
            end
        )
    end
    # ===================== Stage objective =====================
    if stage == 1
        @stageobjective(subproblem, 0.0)
    else
        @stageobjective(
            subproblem,
            1000 * (sum(pen_p) + sum(pen_n)) +
            # cost of growing
            C[stage-1, weather] * acres.in +
            sum(
                # inventory cost
                V[stage-1] * bales[cutting].in * t[stage-1] +
                # purchase cost
                r[cutting][stage-1, weather] * buy[cutting] +
                # feed cost
                S[cutting, stage-1] * eat[cutting] -
                # sell reward
                q[cutting][stage-1, weather] * sell[cutting] for
                cutting in 1:3
            )
        )
    end
    return
end
SDDP.train(model, iteration_limit = 5)
# @test SDDP.termination_status(model) == :simulation_stopping
# @test SDDP.calculate_bound(model) ≈ 4074.1391 atol = 1e-5




-------------------------------------------------------------------
         SDDP.jl (c) Oscar Dowson and contributors, 2017-24
-------------------------------------------------------------------
problem
  nodes           : 10
  state variables : 4
  scenarios       : 2.70000e+01
  existing cuts   : false
options
  solver          : serial mode
  risk measure    : SDDP.Expectation()
  sampling scheme : SDDP.InSampleMonteCarlo
subproblem structure
  VariableRef                             : [24, 24]
  AffExpr in MOI.EqualTo{Float64}         : [3, 3]
  AffExpr in MOI.GreaterThan{Float64}     : [1, 1]
  AffExpr in MOI.LessThan{Float64}        : [1, 6]
  VariableRef in MOI.GreaterThan{Float64} : [20, 20]
  VariableRef in MOI.LessThan{Float64}    : [1, 2]
numerical stability report
  matrix range     [1e+00, 8e+01]
  objective range  [1e+00, 1e+03]
  bounds range     [6e+01, 6e+01]
  rhs range        [2e+02, 3e+03]
-------------------------------------------------------------------
 iterati

In [174]:
print(model[(3,2)].subproblem)



In [117]:
simulations = SDDP.simulate(
    # The trained model to simulate.
    model,
    # The number of replications.
    10,
    # A list of names to record the values of.
    [:acres,:bales],
)



10-element Vector{Vector{Dict{Symbol, Any}}}:
 [Dict(:bales => SDDP.State{Float64}[SDDP.State{Float64}(0.0, -0.0), SDDP.State{Float64}(0.0, -0.0), SDDP.State{Float64}(0.0, -0.0)], :bellman_term => 4074.1391000000003, :noise_term => nothing, :node_index => (1, 1), :stage_objective => 0.0, :objective_state => nothing, :belief => Dict((1, 1) => 1.0), :acres => SDDP.State{Float64}(60.0, 42.79999999999997)), Dict(:bales => SDDP.State{Float64}[SDDP.State{Float64}(-0.0, 3000.0), SDDP.State{Float64}(-0.0, 0.0), SDDP.State{Float64}(-0.0, 0.0)], :bellman_term => 1120.4041000000007, :noise_term => nothing, :node_index => (2, 2), :stage_objective => 2139.9999999999986, :objective_state => nothing, :belief => Dict((2, 2) => 1.0), :acres => SDDP.State{Float64}(42.79999999999997, 42.79999999999997)), Dict(:bales => SDDP.State{Float64}[SDDP.State{Float64}(3000.0, 0.0), SDDP.State{Float64}(-0.0, 0.0), SDDP.State{Float64}(0.0, 0.0)], :bellman_term => 3480.4041000000016, :noise_term => nothing, :node_ind

In [164]:
replication = 3
stage = 2
simulations[replication][stage]

Dict{Symbol, Any} with 8 entries:
  :bales           => SDDP.State{Float64}[State{Float64}(-0.0, 1074.0), State{F…
  :bellman_term    => 4009.4
  :noise_term      => nothing
  :node_index      => (2, 1)
  :stage_objective => 2140.0
  :objective_state => nothing
  :belief          => Dict((2, 1)=>1.0)
  :acres           => State{Float64}(42.8, 42.8)